<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Описание-проекта" data-toc-modified-id="Описание-проекта-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Описание проекта</a></span></li><li><span><a href="#Изучаем-данные" data-toc-modified-id="Изучаем-данные-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Изучаем данные</a></span></li><li><span><a href="#Предобработка-данных" data-toc-modified-id="Предобработка-данных-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Предобработка данных</a></span></li><li><span><a href="#Векторизация" data-toc-modified-id="Векторизация-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Векторизация</a></span></li><li><span><a href="#Обучение-моделей" data-toc-modified-id="Обучение-моделей-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Обучение моделей</a></span></li><li><span><a href="#Общие-выводы" data-toc-modified-id="Общие-выводы-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Общие выводы</a></span></li></ul></div>

# Проект: определение токсичных комментариев

## Описание проекта

Интернет-магазин запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

**Цель работы:**  

1. Провести предобработку данных: лемматизировать текст, убрать стоп-слова и прочие символы, и тд 

2. Обучить модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.

3. Постройте модель со значением метрики качества *F1* не меньше 0.75. 

## Изучаем данные

Устанавливаем необходимые библиотеки.

In [1]:
# !pip install spacy
# !python -m spacy download en_core_web_sm
# !pip install pymystem3
# !pip install xgboost

Импортируем необходимые библиотеки

In [2]:
import pandas as pd
import numpy as np
import spacy
from spacy.lang.en.stop_words import STOP_WORDS

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

import lightgbm
from catboost import CatBoostClassifier

Сохраняем датасет в переменной data.

In [3]:
try: 
    data = pd.read_csv('/datasets/toxic_comments.csv')
except:
    data = pd.read_csv(r'C:\Users\Хозяйка\Проекты Практикума\datasets\toxic_comments.csv')

In [4]:
data.head(10)

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0
5,5,"""\n\nCongratulations from me as well, use the ...",0
6,6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,7,Your vandalism to the Matt Shirvington article...,0
8,8,Sorry if the word 'nonsense' was offensive to ...,0
9,9,alignment on this subject and which are contra...,0


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


**Наблюдения:**
- Корпус текстов на английском языке;
- В данных содержится колонка с дублированными индексами, которую нужно будет удалить;
- Пропущенных значений нет;

## Предобработка данных

Сразу **удаляем колонку 'Unnamed: 0'**. Скорее всего она добавилась в датасет из-за неправильной выгрузки информации.

In [7]:
data = data.drop('Unnamed: 0', axis=1)
data.columns

Index(['text', 'toxic'], dtype='object')

Теперь **создадим фукцию**, которая будет:  
- чистить текст от ненужных символов;
- удалит стоп-слова из текста;
- лемматизирует текст;
- приведет слова к нижнему регистру;

In [8]:
# создаем модель, которая будет разбивать текст на токены

nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

In [9]:
def clearing_n_lemmatizing(text):
    
    clear_text = []
    doc = nlp(text)
    
    # перебираем токены
    for token in doc:
        
        # оставляем только слова, исключая прочие символы
        if token.is_alpha == True:
            
            # исключаем стоп-слова
            if token.is_stop == False:
                
                # сохраняем в список лемматизированные слова и приводим их к нижнему регистру
                clear_text.append(token.lemma_.lower())
    
    # объединяем список из очищенных слов в единую строку и выводим ее
    return ' '.join(clear_text)

In [10]:
# проверяем

clearing_n_lemmatizing(data.text[0])

'explanation edit username hardcore metallica fan revert vandalism closure gas vote new york dolls fac remove template talk page retire'

Теперь **создадим новую колонку** в датафрейме с очищенными строками.

In [11]:
%%time

data['lemma'] = data.text.apply(lambda x: clearing_n_lemmatizing(x))

CPU times: total: 32min 33s
Wall time: 49min 50s


In [12]:
# проверяем

data.lemma.head()

0    explanation edit username hardcore metallica f...
1    match background colour seemingly stick thank ...
2    hey man try edit war guy constantly remove rel...
3    real suggestion improvement wonder section sta...
4                        sir hero chance remember page
Name: lemma, dtype: object

Разделим наши данные на признаки и таргет, а затем поделим их на обучающую, валидационную и тестовую выборки в соотношении 3:1:1

In [33]:
x = data.lemma
y = data.toxic

In [34]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [35]:
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.25)

In [36]:
x_train.shape[0], x_valid.shape[0], x_test.shape[0]

(95574, 31859, 31859)

**Итоги:**

- Мы очистили комментарии от лишних символов и стоп-слов;
- Лемматизировали каждое слово;
- Привели слова к нижнему регистру, чтобы слова написанные с заглавной буквы и нет обрабатывались машиной одинаково;
- Сохранили в отдельные переменные признаки и таргет;

## Векторизация

Мы будем использовать две технологии приведения данных в векторный вид: TF-IDF векторизация и с помощью гриппировки. Используем трансформеры **TfidfVectorizer** и **CountVectorizer**

Транформируем отдельно с помощью этих трансформеров тренировочную и тестовую выборку и сохраним в новые переменные. Добавим атрибут *stop_words*, чтобы трансформер дополнительно проверил наличие стоп-слов в текстах.

In [37]:
tfidf = TfidfVectorizer(stop_words='english', dtype=np.float32)
counter = CountVectorizer(stop_words='english', dtype=np.float32)

In [38]:
%%time

# трансформируем данные

x_train_tfidf = tfidf.fit_transform(x_train)
x_valid_tfidf = tfidf.transform(x_valid)
x_test_tfidf = tfidf.transform(x_test)

CPU times: total: 7.09 s
Wall time: 12.9 s


In [39]:
x_train_tfidf

<95574x109455 sparse matrix of type '<class 'numpy.float32'>'
	with 2080537 stored elements in Compressed Sparse Row format>

In [40]:
%%time

x_train_counted = counter.fit_transform(x_train)
x_valid_counted = counter.transform(x_valid)
x_test_counted = counter.transform(x_test)

CPU times: total: 7.31 s
Wall time: 12.7 s


In [41]:
x_train_counted

<95574x109455 sparse matrix of type '<class 'numpy.float32'>'
	with 2080537 stored elements in Compressed Sparse Row format>

## Обучение моделей

Теперь подготовим модели к обучению на наших векторизованных данных. Будем использовать:  
- **LogisticRegression**
- **LGBMClassifier**
- **RidgeClassifier**

У всех моделей привоим атрибуту 'class_weight' значение 'balanced', потому что кол-во положительных и отрицательных объектов у нас различается.

Проверяем первую модель: **LogisticRegression**

In [42]:
%%time
model_log1 = LogisticRegression(solver='lbfgs', class_weight='balanced')
model_log1.fit(x_train_tfidf, y_train)
pred = model_log1.predict(x_valid_tfidf)
f1_tfidf = f1_score(pred, y_valid)

CPU times: total: 15.4 s
Wall time: 6.57 s


In [43]:
%%time
model_log2 = LogisticRegression(solver='lbfgs', class_weight='balanced')
model_log2.fit(x_train_counted, y_train)
pred = model_log2.predict(x_valid_counted)
f1_counted = f1_score(pred, y_valid)

CPU times: total: 14.9 s
Wall time: 7.14 s


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [44]:
print(f'Результаты модели LogisticRegression:')
print()
print(f'С векторизацией TF-IDF: {f1_tfidf}')
print(f'C группирующей векторизацией: {f1_counted}')

Результаты модели LogisticRegression:

С векторизацией TF-IDF: 0.747752808988764
C группирующей векторизацией: 0.7583872866391995


**LightGBMClassifier**

In [45]:
%%time
model_lgbm1 = lightgbm.LGBMClassifier(class_weight='balanced')
model_lgbm1.fit(x_train_tfidf, y_train)
pred = model_lgbm1.predict(x_valid_tfidf)
f1_tfidf = f1_score(pred, y_valid)

[LightGBM] [Info] Number of positive: 9766, number of negative: 85808
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 3.628084 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 395941
[LightGBM] [Info] Number of data points in the train set: 95574, number of used features: 7917
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
CPU times: total: 1min 12s
Wall time: 34.6 s


In [46]:
%%time
model_lgbm2 = lightgbm.LGBMClassifier(class_weight='balanced')
model_lgbm2.fit(x_train_counted, y_train)
pred = model_lgbm2.predict(x_valid_counted)
f1_counted = f1_score(pred, y_valid)

[LightGBM] [Info] Number of positive: 9766, number of negative: 85808
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 3.700116 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 34709
[LightGBM] [Info] Number of data points in the train set: 95574, number of used features: 7917
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
CPU times: total: 35 s
Wall time: 16.6 s


In [47]:
print(f'Результаты модели LGBMClassifier:')
print()
print(f'С векторизацией TF-IDF: {f1_tfidf}')
print(f'C группирующей векторизацией: {f1_counted}')

Результаты модели LGBMClassifier:

С векторизацией TF-IDF: 0.734468208933508
C группирующей векторизацией: 0.7355251672970614


**CatBoostClassifier**

In [48]:
%%time

model_cat1 = CatBoostClassifier(loss_function='Logloss', 
                           max_depth=8,
                           auto_class_weights='Balanced', 
                           early_stopping_rounds=50)
model_cat1.fit(x_train_tfidf, y_train)
pred = model_cat1.predict(x_valid_tfidf)
f1_tfidf = f1_score(pred, y_valid)

Learning rate set to 0.0722
0:	learn: 0.6553314	total: 5.91s	remaining: 1h 38m 22s
1:	learn: 0.6261565	total: 11.5s	remaining: 1h 35m 57s
2:	learn: 0.6029218	total: 16.3s	remaining: 1h 30m 25s
3:	learn: 0.5839183	total: 21s	remaining: 1h 27m 5s
4:	learn: 0.5684311	total: 25.7s	remaining: 1h 25m 10s
5:	learn: 0.5552836	total: 30.2s	remaining: 1h 23m 29s
6:	learn: 0.5448454	total: 34.7s	remaining: 1h 22m 3s
7:	learn: 0.5374844	total: 39.2s	remaining: 1h 20m 57s
8:	learn: 0.5305090	total: 43.7s	remaining: 1h 20m 13s
9:	learn: 0.5221035	total: 48.1s	remaining: 1h 19m 26s
10:	learn: 0.5155732	total: 52.4s	remaining: 1h 18m 34s
11:	learn: 0.5089404	total: 56.8s	remaining: 1h 17m 56s
12:	learn: 0.5046090	total: 1m 1s	remaining: 1h 17m 18s
13:	learn: 0.5002542	total: 1m 5s	remaining: 1h 16m 59s
14:	learn: 0.4960052	total: 1m 9s	remaining: 1h 16m 35s
15:	learn: 0.4906558	total: 1m 14s	remaining: 1h 16m 18s
16:	learn: 0.4847601	total: 1m 18s	remaining: 1h 16m 6s
17:	learn: 0.4811204	total: 1m 23

In [49]:
%%time

model_cat2 = CatBoostClassifier(loss_function='Logloss', 
                           max_depth=8,
                           auto_class_weights='Balanced', 
                           early_stopping_rounds=50)
model_cat2.fit(x_train_counted, y_train)
pred = model_cat2.predict(x_valid_counted)
f1_counted = f1_score(pred, y_valid)

Learning rate set to 0.0722
0:	learn: 0.6559444	total: 1.58s	remaining: 26m 22s
1:	learn: 0.6250174	total: 3.63s	remaining: 30m 11s
2:	learn: 0.6021410	total: 5.44s	remaining: 30m 8s
3:	learn: 0.5835587	total: 7.12s	remaining: 29m 32s
4:	learn: 0.5688074	total: 8.75s	remaining: 29m 1s
5:	learn: 0.5553631	total: 10.4s	remaining: 28m 43s
6:	learn: 0.5452342	total: 12s	remaining: 28m 26s
7:	learn: 0.5375160	total: 13.7s	remaining: 28m 18s
8:	learn: 0.5301630	total: 15.4s	remaining: 28m 11s
9:	learn: 0.5229705	total: 17s	remaining: 28m 3s
10:	learn: 0.5156218	total: 18.7s	remaining: 27m 58s
11:	learn: 0.5107326	total: 20.3s	remaining: 27m 52s
12:	learn: 0.5072794	total: 21.9s	remaining: 27m 46s
13:	learn: 0.5027498	total: 23.6s	remaining: 27m 40s
14:	learn: 0.4980340	total: 25.2s	remaining: 27m 35s
15:	learn: 0.4948584	total: 26.9s	remaining: 27m 32s
16:	learn: 0.4912264	total: 28.5s	remaining: 27m 28s
17:	learn: 0.4875903	total: 30.1s	remaining: 27m 24s
18:	learn: 0.4831349	total: 31.8s	r

In [50]:
print(f'Результаты модели CatBoostRegressor:')
print()
print(f'С векторизацией TF-IDF: {f1_tfidf}')
print(f'C группирующей векторизацией: {f1_counted}')

Результаты модели CatBoostRegressor:

С векторизацией TF-IDF: 0.760059171597633
C группирующей векторизацией: 0.7500730780473547


Теперь приступим к **тестированию лучших моделей**. У нас это:
- CatBoostClassifier, обученная на данных с TF-IDF векторизацией;
- CatBoostClassifier, обученная на данных с CountVectorizer;
- LogisticRegression, обученная на данных с CountVectorizer;

Все они подходят по параметру f1 > 0.75. Выберем ту, у которой будет результат лучше на тестовой выборке.

In [54]:
print(f'Оценка предсказаний CatBoostClassifier (TfidfVectorizer): {f1_score(model_cat1.predict(x_test_counted), y_test)}')
print(f'Оценка предсказаний CatBoostClassifier (CountVectorizer): {f1_score(model_cat2.predict(x_test_counted), y_test)}')
print(f'Оценка предсказаний LogisticRegression (CountVectorizer): {f1_score(model_log2.predict(x_test_counted), y_test)}')

Оценка предсказаний CatBoostClassifier (TfidfVectorizer): 0.7270745333151655
Оценка предсказаний CatBoostClassifier (CountVectorizer): 0.7472092694644624
Оценка предсказаний LogisticRegression (CountVectorizer): 0.7585911877940967


## Общие выводы

Мы проверили три модели для работы с категориальным целевым признаком. **Итоговые наблюдения**:  

1) Модели обучаются быстрее на данных с CountVectorizer векторизацией. Однако практически во всех случаях качество предсказаний хуже, чем у TfIdfVectorizer.  

2) Модель LogisticRegression - единственное исключение, потому что на ней векторизация CountVectorizer показала результат лучше, чем вторая. Также у нее лучший результат метрики F1 и самое быстрое время обучения.  

3) Модель LightGBM Classifier имеет самый низкий результат по необходимой нам метрике. Он не сильно низкий, но не подходит нам по условию задачи (f1 > 0.75). Время обработки информации приемлемое.  

4) CatBoostClassifier. Обе ее модели, обученные на данных с разной векторизацией, подходят нам по условию. У них достаточно высокий показатель f1-метрики, но обе имеют очень большое время обучения. Особенно та, которая училась на данных с TfIdfVectorizer векторизацией (почти 2 часа).  

5) Мы проверили все три модели, которые показали удовлетворительный результат на валидационной выборке, были проверены на тестовой выборке. Единственная, у которой практически не упал показатель f1-метрики - это Логистическая регрессия. У остальных сильно упал результат.
 
Исходя из этих результатов, **рекомендую к использованию модель LogisticRegression со сбалансированными классами**.